In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# Added base64 to handle logo
import base64

#### FIX ME #####
# Updated animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME updated with my username/password and CRUD Python module name
username = "aacuser"
password = "passwordPrompt()"
shelter = AnimalShelter(username, password)

# Class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Added in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Placed the HTML image tag in the line below into the app.layout code 
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
#FIX ME Added "Developed by Quinnie Ho at Global Rain" as unique identifier

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.A([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={"height": "2" "00px"})], # Centered and resized logo
        href="https:www.snhu.edu" # Added URL logo tag to client's home page
    )),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))), # Updated name of dashboard
    html.Hr(),
    html.Div(
#FIXME Added in code for the button interactive filtering options
    className='row',
        style={'display': 'flex'},
            children=[
                html.Button(id='button_one', n_clicks=0, children='Water Rescue'),
                html.Button(id='button_two', n_clicks=0, children='Mountain Rescue'),
                html.Button(id='button_three', n_clicks=0, children='Disaster Rescue'),
                html.Button(id='button_four', n_clicks=0, children='Reset')          
        ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for the interactive data table to make it user-friendly for the client
        editable=False,
        filter_action="native",
        page_size=10,
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        style_table={'overflowX': 'auto'},
        style_header={
            'backgroundColor': 'black',
            'color': 'white',
            'fontWeight': 'bold'},
        style_cell={
            'whiteSpace': 'normal',
            'height': 'auto',
            'minWidth': '160px', 'width': '160px', 'maxWidth': '160px'}
    ),
    html.Br(),
    html.Hr(),
# This sets up the dashboard so that the pie chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6'
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),

    # Unique identifier
    html.Header("Developed by Quinnie Ho at Global Rain")
])



#############################################
# Interaction Between Components / Controller
#############################################
 
@app.callback([Output('datatable-id', "data"), 
              Output('datatable-id','columns')],
              [Input('button_one', 'n_clicks'),
              Input('button_two', 'n_clicks'),
              Input('button_three', 'n_clicks'),
              Input('button_four', 'n_clicks')])
def update_dashboard(button1, button2, button3, button4):
### FIX ME Added code to filter interactive data table with MongoDB queries
    
    # Declare/initialize local variables
    int(button1, button2, button3, button4)
    dashboard_is_open = true
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')    
    
    while dashboard_is_open:
                         
        if (button1 == 1): 
            df = pd.DataFrame.from_records(shelter.read(
                {'$and': [
                {'$or': [
                    {"breed": "Labrador Retriever Mix"},
                    {"breed": "Newfoundland"},
                    {"breed": "Chesapeake Bay Retriever"}
                ]},
                {"sex_upon_outcome":"Intact Female"},
                {"age_upon_outcome_in_weeks": {'$gte':26, '$lte':156}}
                ]}
            ))
            return (data,columns)
            button1 = 0
            continue


        if (button2 == 1):         
            df = pd.DataFrame.from_records(shelter.read(
                {'$and': [ 
                {'$or': [
                    {'breed': 'German Shepherd'},
                    {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'Siberian Husky'},
                    {'breed': 'Rottweiler'}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {'$gte':26, '$lte':156}}
                ]}
            ))
            return (data,columns)
            button2 = 0
            continue


        if (button3 == 1): 
            df = pd.DataFrame.from_records(shelter.read(
                {'$and': [ 
                {'$or': [
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Golden Retriever'},
                    {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'}
                ]}, 
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {'$gte':20, '$lte':300}}
                ]}
            ))
            return (data,columns)
            button3 = 0
            continue
            
        if (button4 == 1):
            df = pd.DataFrame.from_records(shelter.read({}))
            return (data,columns)
            button4 = 0
            continue
            

  


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"), 
    [Input("datatable_id", "derived_viewport_data")]
)
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
    #dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(df, values="animal_id", names="outcome_type", title="Results by Rescue Type")
        )
    ]




@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
#FIXME: Added in the code for geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(
                    position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']],
                    children=[
                        dl.Tooltip(dff.loc[0, 'breed']),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.loc[0, 'name'])
                        ])
                    ]
                )
            ]
        )
    ]


app